# Maximum Drawdown and Calmar Ratio

## Getting ready

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("seaborn")

In [ ]:
returns = pd.read_csv("returns.csv", index_col = "Date", parse_dates = ["Date"])
returns

In [ ]:
returns.info()

In [ ]:
returns.cumsum().apply(np.exp).plot(figsize = (12, 8))
plt.show()

## Maximum Drawdown

In [ ]:
symbol = "USD_GBP"

In [ ]:
returns[symbol].cumsum().apply(np.exp).plot(figsize = (12, 8))
plt.show()

In [ ]:
instr = returns[symbol].to_frame().copy()
instr

In [ ]:
instr["creturns"] = instr.cumsum().apply(np.exp) # cumulative returns (normalized prices with Base == 1)

In [ ]:
instr["cummax"] = instr.creturns.cummax() # cumulative maximum of creturns

In [ ]:
instr

In [ ]:
instr[["creturns", "cummax"]].plot(figsize = (15, 8), fontsize = 13)
plt.legend(fontsize = 13)
plt.show()

In [ ]:
instr["drawdown"] = -(instr["creturns"] - instr["cummax"]) / instr["cummax"] # (pos.) drawdown (in %)
instr

In [ ]:
instr[["creturns", "cummax", "drawdown"]].plot(figsize = (15, 8), fontsize = 13, secondary_y = "drawdown")
plt.legend(fontsize = 13)
plt.show()

In [ ]:
max_drawdown = instr.drawdown.max() # maximum drawdown
max_drawdown

In [ ]:
instr.drawdown.idxmax() # maximum drawdown date 

In [ ]:
instr.loc[instr.drawdown.idxmax()]

In [ ]:
(0.941169 - 1.127116) / 1.127116

## Calmar Ratio

In [ ]:
max_drawdown

In [ ]:
cagr = np.exp(instr[symbol].sum())**(1/((instr.index[-1] - instr.index[0]).days / 365.25)) - 1 
cagr

In [ ]:
calmar = cagr / max_drawdown
calmar

## Max Drawdown Duration

In [ ]:
instr

In [ ]:
instr[["creturns", "cummax", "drawdown"]].plot(figsize = (15, 8), fontsize = 13, secondary_y = "drawdown")
plt.legend(fontsize = 13)
plt.show()

In [ ]:
drawdown = instr.drawdown.copy()
drawdown

- Drawdown Period: Time Period between peaks 
- recall: whenever drawdown == 0, a new peak has been reached

In [ ]:
begin = drawdown[drawdown == 0].index # get all peak dates (beginning of Drawdown periods)
begin

In [ ]:
end = begin[1:] # get the corresponding end dates for all Drawdown periods
end = end.append(pd.DatetimeIndex([drawdown.index[-1]])) # add last available date
end

In [ ]:
periods = end - begin # time difference between peaks
periods

In [ ]:
max_ddd = periods.max() # max drawdown duration
max_ddd

In [ ]:
max_ddd.days

## Putting everything together

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
returns = pd.read_csv("returns.csv", index_col = "Date", parse_dates = ["Date"])
returns

In [ ]:
def max_drawdown(series):
    creturns = series.cumsum().apply(np.exp)
    cummax = creturns.cummax()
    drawdown = (cummax - creturns)/cummax
    max_dd = drawdown.max()
    return max_dd

In [ ]:
returns.apply(max_drawdown).sort_values()

In [ ]:
def calculate_cagr(series):
    return np.exp(series.sum())**(1/((series.index[-1] - series.index[0]).days / 365.25)) - 1

In [ ]:
def calmar(series):
    
    max_dd = max_drawdown(series)
    if max_dd == 0:
        return np.nan
    else:
        cagr = calculate_cagr(series)
        calmar = cagr / max_dd
        return calmar

In [ ]:
returns.apply(calmar).sort_values(ascending = False)

In [ ]:
def max_dd_duration(series):
    creturns = series.cumsum().apply(np.exp)
    cummax = creturns.cummax()
    drawdown = (cummax - creturns)/cummax
    
    begin = drawdown[drawdown == 0].index
    end = begin[1:]
    end = end.append(pd.DatetimeIndex([drawdown.index[-1]]))
    periods = end - begin
    max_ddd = periods.max()
    return max_ddd.days   

In [ ]:
returns.apply(max_dd_duration).sort_values()

-----------------------

## Coding Challenge

__Calculate and compare__ <br>
- __Maximum Drawdown__
- __Calmar Ratio__
- __Maximum Drawdown Duration__ <br>

for __30 large US stocks__ that currently form the Dow Jones Industrial Average Index ("Dow Jones") for the time period between April 2019 and June 2021.

__Hint:__ You can __import__ the price data from __"Dow_Jones.csv"__.
 

Determine the __best performing stock__ and the __worst performing stock__ according to the Calmar Ratio.

__Compare__ Calmar Ratio and Sharpe Ratio. Does the __ranking change__?

(Remark: Dividends are ignored here. Hence, for simplicity reasons, the Calmar Ratio is based on Price Returns only. As a consequence, dividend-paying stocks are getting penalized.) 

In [106]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn')
%matplotlib inline

In [160]:
raw      = pd.read_csv('Dow_Jones.csv', parse_dates=['Date'], index_col='Date')
returns  = pd.DataFrame()

logr     = np.log(raw / raw.shift(1)); logr.dropna(inplace=True)
logrc    = logr.cumsum().apply(np.exp)
logrcm   = logr.cummax()
simp     = raw / raw.shift(1); simp.dropna(inplace=True); simp
simpcum  = simp.cumprod()
simpcumm = simpcum.cummax()
td_years = (logrc.index[-1] - logrc.index[0]).days / 365.25
cagr     = (logrc.iloc[-1]/logrc.iloc[0])**(1/td_years)-1
drawdown = (-(simpcum - simpcumm)/simpcumm)

#simpcumm.plot(figsize=(15,8)); simpcum.plot(figsize=(15,8))
#plt.legend()
plt.show()
max_dd_date = drawdown.idxmax()
begin = drawdown[drawdown==0].index
end = begin[1:].append(pd.DatetimeIndex([drawdown.index[-1]]))
periods = (end - begin)
calmar = (cagr/drawdown.max()).to_frame().sort_values(by=0, ascending=False)
calmar
max_ddd = periods.max(); max_ddd
drawdown.cummax().iloc[-1].sort_values().to_frame()

,2021-06-14
WMT,0.165325
VZ,0.195425
PG,0.231556
AMGN,0.250658
JNJ,0.278265
MRK,0.278574
MSFT,0.282353
AAPL,0.314273
INTC,0.355776
CRM,0.357158


## +++ Please stop here in case you don´t want to see the solution!!! +++++

## Coding Challenge Solution

In [140]:
import pandas as pd
import numpy as np

In [141]:
df = pd.read_csv("Dow_Jones.csv", index_col = "Date", parse_dates = ["Date"])
df

,AAPL,AMGN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,DOW,...,MRK,MSFT,NKE,PG,TRV,UNH,V,VZ,WBA,WMT
Date,,,,,,,,,,,,,,,,,,,,,
2019-04-01,47.810001,191.770004,111.699997,391.540009,140.250000,161.179993,54.980000,124.940002,112.510002,53.500000,...,79.484734,119.019997,85.230003,103.639999,138.020004,245.539993,157.259995,59.090000,63.490002,97.820000
2019-04-02,48.505001,192.429993,111.000000,390.750000,140.190002,160.509995,55.290001,124.930000,111.959999,56.250000,...,79.398857,119.190002,84.370003,103.750000,136.809998,244.410004,157.779999,58.529999,55.360001,96.940002
2019-04-03,48.837502,192.919998,110.559998,384.739990,139.259995,162.619995,55.630001,123.889999,112.519997,56.880001,...,79.370232,119.970001,84.470001,103.690002,136.600006,245.839996,158.460007,58.869999,54.840000,97.190002
2019-04-04,48.922501,192.330002,110.120003,395.859985,140.130005,158.520004,55.139999,124.800003,114.750000,59.709999,...,78.101143,119.360001,85.279999,103.430000,136.520004,247.389999,157.639999,58.990002,54.150002,98.110001
2019-04-05,49.250000,195.410004,110.959999,391.929993,140.360001,158.559998,55.209999,126.419998,115.000000,57.240002,...,77.433205,119.889999,85.400002,103.650002,136.399994,248.779999,157.649994,59.090000,54.689999,98.830002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-08,126.739998,236.820007,166.509995,252.759995,240.160004,236.419998,54.130001,108.730003,176.330002,69.379997,...,72.400002,252.570007,133.350006,134.839996,157.539993,401.739990,232.050003,57.150002,54.000000,139.830002
2021-06-09,127.129997,239.520004,164.089996,248.059998,234.649994,236.089996,54.020000,107.779999,176.039993,68.839996,...,74.040001,253.589996,131.839996,134.789993,154.740005,401.100006,232.309998,57.049999,53.430000,139.080002
2021-06-10,126.110001,244.639999,162.229996,248.339996,225.729996,240.190002,55.029999,108.410004,176.570007,67.989998,...,76.139999,257.239990,130.979996,135.789993,154.020004,401.489990,233.949997,57.340000,55.310001,139.880005


In [142]:
returns = np.log(df / df.shift()) # log returns
returns

,AAPL,AMGN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,DOW,...,MRK,MSFT,NKE,PG,TRV,UNH,V,VZ,WBA,WMT
Date,,,,,,,,,,,,,,,,,,,,,
2019-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-04-02,0.014432,0.003436,-0.006286,-0.002020,-0.000428,-0.004165,0.005623,-0.000080,-0.004900,0.050124,...,-0.001081,0.001427,-0.010142,0.001061,-0.008806,-0.004613,0.003301,-0.009522,-0.137025,-0.009037
2019-04-03,0.006832,0.002543,-0.003972,-0.015500,-0.006656,0.013060,0.006131,-0.008360,0.004989,0.011138,...,-0.000361,0.006523,0.001185,-0.000578,-0.001536,0.005834,0.004301,0.005792,-0.009437,0.002576
2019-04-04,0.001739,-0.003063,-0.003988,0.028493,0.006228,-0.025535,-0.008847,0.007318,0.019625,0.048556,...,-0.016119,-0.005098,0.009543,-0.002511,-0.000586,0.006285,-0.005188,0.002036,-0.012662,0.009421
2019-04-05,0.006672,0.015887,0.007599,-0.009977,0.001640,0.000252,0.001269,0.012897,0.002176,-0.042247,...,-0.008589,0.004431,0.001406,0.002125,-0.000879,0.005603,0.000063,0.001694,0.009923,0.007312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-08,0.006650,-0.001561,0.003550,0.000396,0.001667,-0.007249,0.003887,0.009054,-0.003736,0.002020,...,-0.005510,-0.004897,-0.004489,-0.011649,-0.001396,0.003716,0.003151,-0.000874,-0.010317,-0.007623
2021-06-09,0.003072,0.011337,-0.014640,-0.018770,-0.023210,-0.001397,-0.002034,-0.008776,-0.001646,-0.007814,...,0.022399,0.004030,-0.011388,-0.000371,-0.017933,-0.001594,0.001120,-0.001751,-0.010612,-0.005378
2021-06-10,-0.008056,0.021151,-0.011400,0.001128,-0.038755,0.017217,0.018524,0.005828,0.003006,-0.012424,...,0.027968,0.014291,-0.006544,0.007392,-0.004664,0.000972,0.007035,0.005070,0.034581,0.005736


__Maximum Drawdown__

In [143]:
def max_drawdown(series):
    creturns = series.cumsum().apply(np.exp)
    cummax = creturns.cummax()
    drawdown = (cummax - creturns)/cummax
    max_dd = drawdown.max()
    return max_dd

In [144]:
returns.apply(max_drawdown).sort_values()

WMT     0.165325
VZ      0.195425
PG      0.231556
AMGN    0.250658
JNJ     0.278265
MRK     0.278574
MSFT    0.282353
AAPL    0.314273
INTC    0.355776
CRM     0.357158
UNH     0.361763
V       0.363649
KO      0.375353
MCD     0.380059
HD      0.384058
CAT     0.389823
IBM     0.395445
NKE     0.399503
CSCO    0.428079
HON     0.433171
DIS     0.434450
JPM     0.439861
GS      0.459515
MMM     0.463007
WBA     0.465561
TRV     0.472389
AXP     0.496385
CVX     0.571992
DOW     0.631552
BA      0.759991
dtype: float64

__Calmar Ratio__

In [ ]:
def calculate_cagr(series):
    return np.exp(series.sum())**(1/((series.index[-1] - series.index[0]).days / 365.25)) - 1

In [ ]:
def calmar(series):
    
    max_dd = max_drawdown(series)
    if max_dd == 0:
        return np.nan
    else:
        cagr = calculate_cagr(series)
        calmar = cagr / max_dd
        return calmar

In [ ]:
calm = returns.apply(calmar).sort_values(ascending = False)
calm

Best Performing Stock: __Apple__ (AAPL) <br>
Worst Performing Stock: __Non-determinable__ (note: you can´t compare negative Calmar Ratios)

__Maximum Drawdown Duration__

In [ ]:
def max_dd_duration(series):
    creturns = series.cumsum().apply(np.exp)
    cummax = creturns.cummax()
    drawdown = (cummax - creturns)/cummax
    
    begin = drawdown[drawdown == 0].index
    end = begin[1:]
    end = end.append(pd.DatetimeIndex([drawdown.index[-1]]))
    periods = end - begin
    max_ddd = periods.max()
    return max_ddd.days 

In [ ]:
returns.apply(max_dd_duration).sort_values()

In [ ]:
def sharpe(series, rf = 0):
    
    if series.std() == 0:
        return np.nan
    else:
        return (series.mean() - rf) / series.std() * np.sqrt(series.count() / ((series.index[-1] - series.index[0]).days / 365.25))

In [ ]:
sha = returns.apply(sharpe).sort_values(ascending = False)
sha

In [ ]:
merged = pd.concat([calm, sha], axis = 1)
merged

In [ ]:
merged.columns = ["Calmar", "Sharpe"]

In [ ]:
merged.rank(ascending = False)

-> Some Differences. __Salesforce (CRM) gets better ranked__ with Calmar (-4) while __The Nike gets penalized__ by Calmar (+5).